# Database

In [ ]:
//| export

import { Database } from "@db/sqlite";


## Bootstrap database

In [ ]:
//| export

const tables: Record<string, string> = {
  "foo": "CREATE TABLE foo (id INTEGER PRIMARY KEY, name TEXT)",
  "bar": "CREATE TABLE bar (id INTEGER PRIMARY KEY, name TEXT)",
};

export const getDatabase = () => {
  const db = new Database(Deno.env.get("DB_URL") || ":memory:");
  const existingTables = db
    .prepare("SELECT name FROM sqlite_master WHERE type='table'")
    .all<{ name: string }>();

  Object.keys(tables).forEach((table) => {
    if (!existingTables.some((t) => t.name === table)) {
      db.prepare(tables[table]).run();
    }
  });

  return db;
};

In [ ]:
import { assert } from "asserts";

Deno.test("getDatabase", () => {
  const db = getDatabase();
  const ts = db.prepare("SELECT name FROM sqlite_master WHERE type='table'")
    .all<{
      name: string;
    }>();
  assert(ts.length === 2);
  assert(ts.some((t) => t.name === "foo"));
  assert(ts.some((t) => t.name === "bar"));
});